In [1]:
import numpy as np
import pandas as pd
import random
import operator
import pandas as pd
import matplotlib.pyplot as plt

# Reading and Preprocessing Data

In [2]:
def read_excel(data):
    df = pd.read_excel('Data Prep.xlsx', index_col=0, sheet_name=data)
    return df

def read_excel_revisi(data):
    df = pd.read_excel('Data Prep Revisi.xlsx', index_col=0, sheet_name=data)
    return df



def mirroring(df): 
    df_c = df.copy()
    df = df.fillna(0)
    df = df + df.T
    df[df_c.isnull() & (df == 0)] = None
    return df

def set_same_zero(df):
    df_column = df.columns.values
    df_column = set(df_column)
    df_index = df.index.values
    df_index = set(df_index)
    same_port = df_index & df_column

    # making same port data into 0 
    for i in same_port:
        df[i][i]=0
    return df


In [3]:
alpha_up = read_excel('alpha_up')

GAMMA_TL = read_excel('GAMMA_TL')

w = read_excel('w')
w = mirroring(w)
w = set_same_zero(w)

TL_char = read_excel_revisi('TL_char')
PL_char = read_excel_revisi('PL_char')
PR_char = read_excel_revisi('PR_char')

# TL_char = read_excel_revisi('TL_char')
# PL_char = read_excel_revisi('PL_char')
# PR_char = read_excel_revisi('PR_char')

beta_up = read_excel('beta_up')

gamma_uu = read_excel('gamma_uu')
gamma_uu = mirroring(gamma_uu)
gamma_uu = set_same_zero(gamma_uu)


SIGMA_PL = read_excel('SIGMA_PL')



sigma_pr = read_excel('sigma_pr')

PSI_PR = read_excel('PSI_PR')

z_pr = read_excel('z_pr')



port_item = pd.read_excel('data.xlsx',index_col=0 , sheet_name='Barang')

biaya_jarak_teus = read_excel_revisi('Biaya_Jarak_Teus')

In [4]:
def fill_mean(df):
    df = df.fillna(df.mean().mean())
    return df

w = fill_mean(w)
SIGMA_PL = fill_mean(SIGMA_PL)
sigma_pr = fill_mean(sigma_pr)
z_pr = fill_mean(z_pr)
PSI_PR = fill_mean(PSI_PR)
PR_char = fill_mean(PR_char)
biaya_jarak_teus = fill_mean(biaya_jarak_teus)

# Data Dummy

In [5]:
port_item  = pd.read_csv('barang_dummy.csv',index_col=0)

# Creating Objects and Function

In [6]:
def port_items(port):
    global port_item
    item_list = port_item[port_item['Pelabuhan'] == port]
    return item_list

In [7]:
def find_distance(port_a,port_b):
    distance = np.random.rand()*100 #TODO: FIND DISTANCE
    return distance

In [8]:
def choose_char(ship_type):
    global TL_char, PL_char, PR_char
    if ship_type=='TL':
        ship_char = TL_char
    elif ship_type=='PL':
        ship_char = PL_char
    elif ship_type=='PR':
        ship_char = PR_char
    return ship_char

In [9]:
def per_item_revenue(data):
    distance = find_distance(data['Pelabuhan'],data['Tujuan Pelabuhan'])
    Bobot = data['Bobot']
    BJT = biaya_jarak_teus[data['Pelabuhan']][data['Tujuan Pelabuhan']]
    income = distance*Bobot*BJT
    return income

In [10]:
def port_revenue(storage_df):
    total_income = 0
    for i in storage_df.index:
        income = per_item_revenue(storage_df.loc[i])
        total_income += income
    return total_income

In [11]:
def item_left(df):
    return int(df['Bobot'].sum())

In [12]:
class Port:
    def __init__(self, name,port_type):
        self.name = name
        self.port_type = port_type
        self.demand = port_items(name)
        
    
    def __repr__(self):
        if self.port_type == 'U':
            port_type_name = 'Utama'
        elif self.port_type == 'P':
            port_type_name = 'Pengumpul'
        elif self.port_type == 'R':
            port_type_name = 'Pengumpan'
        else:
            port_type_name = 'NOT DEFINED'

        return port_type_name + ":" + self.name +' demand:'
    

In [13]:
class Ship:
    def __init__(self, ship_type, route):
        self.ship_type = ship_type
        self.route = route
        self.route_n = 0
        self.revenue = 0
        self.current_port = route[0]
        self.port_cycle = route

        storage_df = pd.DataFrame(columns = ['Pelabuhan','Bobot','Tujuan Pelabuhan'])
        storage_df.index.name = 'code barang'
        self.storage = storage_df
        self.storage_volume = storage_df['Bobot'].sum()
        
    def update_storage_volume(self):
        self.storage_volume = self.storage['Bobot'].sum()
    def cycle_list(self):
        self.port_cycle = list(self.port_cycle[1:])+[self.port_cycle[0]]
    def move_port(self):
        self.cycle_list()
        self.current_port = self.port_cycle[0]
        
    def move_storage(self,port):
        ship_char = choose_char(self.ship_type)

        # ITEM DROP OFF
        ship_item = self.storage[self.storage['Tujuan Pelabuhan'] == port.name]
        self.storage = pd.concat([self.storage, ship_item]).drop_duplicates(keep=False)
        self.update_storage_volume()

        #CALCULATING SHIP REVENUE
        self.revenue += port_revenue(ship_item)
        #CALCULATING SHIP COST        
        total_cost_bongkar = 0
        total_cost_storage = 0
        total_cost_travel_time = 0
        cost_bongkar_time = 0
        for i in list(ship_item.index):
            item = ship_item.loc[i]
            Bobot = item['Bobot']
            bm_time = ship_char[self.current_port.name ]['bm_time']
            C_bm = ship_char[self.current_port.name ]['C_bm']
            inventory_cost = ship_char[self.current_port.name ]['inventory_cost']
            bm_time = ship_char[self.current_port.name ]['bm_time']
            avg_docking_time = ship_char[self.current_port.name ]['avg_docking_time']
            distance = find_distance(item['Pelabuhan'],self.current_port.name )
            V = ship_char['ship_char']['V']
            VC = ship_char['ship_char']['VC']
            port_storage_time = ship_char[self.current_port.name ]['port_storage_time']
            C_storage = ship_char[self.current_port.name ]['C_storage']

            total_cost_bongkar += Bobot*bm_time*C_bm
            total_cost_storage += Bobot * port_storage_time * C_storage
            total_cost_travel_time += inventory_cost*bm_time*(avg_docking_time*(distance/V))
            cost_bongkar_time += inventory_cost * (Bobot/C_bm)

        travel_cost = total_cost_bongkar+ total_cost_storage+total_cost_travel_time+cost_bongkar_time
        self.revenue -= travel_cost
        
        print('port_revenue : '+str(port_revenue(ship_item))+ ' travel_cost : ' + str(travel_cost))
        

        
        # ITEM DROP ON
        total_cost_bongkar = 0
        total_cost_travel_time=0
        
        for i in self.port_cycle:
            port_item = port.demand[port.demand['Tujuan Pelabuhan'] == i.name]
            port.demand = pd.concat([port.demand, port_item]).drop_duplicates(keep=False)
            self.storage = self.storage.append(port_item)
            self.update_storage_volume()
            

        
        
        

        
    
    def Z(self,*ports):
        if self.ship_type == 'TL':
            df = ZTL
            string = 'ZTL'
            Z = float(df[(df[ports[0].port_type] == ports[0].name) & (df[ports[1].port_type] == ports[1].name)][string].values)

        elif self.ship_type == 'PR':
            df = ZPR
            string = 'ZPR'
            Z = float(df[(df[ports[0].port_type] == ports[0].name) & (df[ports[1].port_type] == ports[1].name)][string].values)

        elif self.ship_type == 'PL':
            Z = float(ZPL[(ZPL['U'] == ports[0].name) & (ZPL['U1'] == ports[1].name) & (ZPL['P'] == ports[2].name)]['ZPL'].values)

        else:
            print('ship_type not recognize')
        return Z
      

In [14]:
Tanjung_Perak = Port('Tanjung Perak', 'U')
Tanjung_Priok = Port('Tanjung Priok', 'U')
Saumlaki = Port('Saumlaki', 'P')
Dobo = Port('Dobo','P')
Namlea = Port('Namlea','P')
route = [Tanjung_Perak,Saumlaki,Tanjung_Priok,Dobo,Namlea,Tanjung_Perak]

In [15]:

TL = Ship('TL',route)
PL = Ship('PL',route)
PR = Ship('PR',route)

In [16]:
available_port = [Tanjung_Perak,Saumlaki,Dobo,Namlea]
available_ship = [TL,PL,PR]

In [17]:
def all_item(available_port,available_ship):
    itemdf = pd.DataFrame(columns = ['Pelabuhan','Bobot','Tujuan Pelabuhan'])
    itemdf.index.name = 'code barang'
    for i in available_port:
        itemdf = itemdf.append(i.demand)
    for i in available_ship:
        itemdf = itemdf.append(i.storage)
    return itemdf

In [18]:
new_port_item = all_item(available_port,available_ship)
remaining = item_left(new_port_item)
while remaining>0:
    TL.move_storage(TL.current_port)
    TL.move_port()
    TL.route_n += 1
    new_port_item = all_item(available_port,available_ship)
    remaining = item_left(new_port_item)
#     print('current port: ' + TL.current_port)
#     print(TL.storage)
#     print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
#     print('remaining: ' + str(remaining))
#     print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('************************************************')
    print('************************************************')
    print('storage: ' + str(TL.storage_volume))
    print('money: ' + str(TL.revenue))
    print('************************************************')
    
print('Ship Income: ' + str(TL.revenue))

port_revenue : 0 travel_cost : 0
************************************************
************************************************
storage: 1384
money: 0
************************************************
port_revenue : 2980557640.594279 travel_cost : 102060074.80931911
************************************************
************************************************
storage: 2504
money: 3518568706.739909
************************************************
port_revenue : 78144481972.43364 travel_cost : 84740352.23105152
************************************************
************************************************
storage: 2400
money: 72265227095.88255
************************************************
port_revenue : 31830920971.890003 travel_cost : 963102452.1558931
************************************************
************************************************
storage: 2416
money: 104850708061.76622
************************************************
port_revenue : 6878510724.507338 travel_c